# DermaFusion – Train on Colab (GPU + hair-removed images)

Train the 7-class skin lesion model on **Google Colab** with:
- **GPU** (T4) for fast training
- **Hair-removed preprocessed images** (no color constancy at train time)

## Before running
1. **Runtime → Change runtime type → GPU** (T4).
2. Have project + data in Colab:
   - **Option A (Drive):** Mount Drive; your folder `DermaFusion/` should contain `src/`, `scripts/`, `configs/`, and `data/` with metadata + preprocessed images.
   - **Option B (GitHub):** Clone repo in cell 2; then upload or copy `data/` (metadata + `preprocessed_hair_removed/images/train`, `val`) into the cloned project.
3. Required data layout:
   - `data/raw/metadata/` – ISIC 2018 CSVs (e.g. `ISIC2018_Task3_Training_GroundTruth.csv`, `*_Validation_*`, `*_Test_*`, `*_LesionGroupings.csv`).
   - `data/preprocessed_hair_removed/images/train/` and `.../val/` – hair-removed images (e.g. `.jpg`).

In [ ]:
# 1) Mount Google Drive (if your DermaFusion project and data are on Drive)
from google.colab import drive
drive.mount("/content/drive")

# Set project root: use the path where DermaFusion lives on Drive or /content after clone
import os
PROJECT_ROOT = "/content/drive/MyDrive/DermaFusion"  # Change if your path is different
if not os.path.isdir(PROJECT_ROOT):
    PROJECT_ROOT = "/content/DermaFusion"  # Fallback for clone-from-GitHub flow
os.chdir(PROJECT_ROOT)
print("Project root:", os.getcwd())

In [ ]:
# 2) If you use GitHub instead of Drive: clone repo then copy data into it.
# Uncomment the two lines below, run, then upload data/ into the cloned project.
# !git clone https://github.com/ManikantaSirumalla/DermaFusion.git /content/DermaFusion
# PROJECT_ROOT = "/content/DermaFusion"; os.chdir(PROJECT_ROOT); print("Project root:", PROJECT_ROOT)

In [ ]:
# 3) Install dependencies (run once per session)
!pip install -q -r requirements.txt
print("Done installing.")

In [ ]:
# 4) Check GPU and data paths
import torch
from pathlib import Path

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))

root = Path(PROJECT_ROOT)
meta = root / "data" / "raw" / "metadata"
train_img = root / "data" / "preprocessed_hair_removed" / "images" / "train"
val_img = root / "data" / "preprocessed_hair_removed" / "images" / "val"
print("Metadata dir exists:", meta.exists())
print("Train images dir exists:", train_img.exists())
print("Val images dir exists:", val_img.exists())
if train_img.exists():
    n_train = len(list(train_img.glob("*.jpg")))
    print(f"Train images (approx): {n_train}")

In [ ]:
# 5) Train with hair-removed images on GPU (no device override = use CUDA)
# Uses batch_size 32, 100 epochs, early stopping. Adjust epochs below if you want a quick run.
!python scripts/train.py \
  data.data.use_preprocessed=true \
  data.data.preprocessed_image_dir=data/preprocessed_hair_removed/images \
  data.data.num_workers=2 \
  data.data.validate_images_on_init=false \
  training.training.epochs=100

In [ ]:
# 6) Optional: copy best checkpoint to Drive so you don't lose it when runtime ends
from pathlib import Path
import shutil

ckpt = Path(PROJECT_ROOT) / "outputs" / "checkpoints" / "best.ckpt"
if ckpt.exists():
    dest = Path("/content/drive/MyDrive/DermaFusion_checkpoint_best.ckpt")
    shutil.copy(ckpt, dest)
    print("Saved checkpoint to", dest)
else:
    print("No best.ckpt found at", ckpt)